In [1]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 
print('WORKING ON', my_key['server'], '\n')

##### COLLECT ITEMS TO Release #####
# use either a starting item to fetch all linked items

# Use a starting item to find linked ones
# starting_items = ['46db06ad-b399-4cf4-9acc-07b3e25ef132']
#add_items = get_query_or_linked(my_key, linked=starting_items)

# or a search query
#my_query = '/search/?q=GOLD&type=Item&limit=all'
#add_items = get_query_or_linked(my_key, query=my_query)

# if you want you can dump them to separate json files (will work as test insert)
# dump_to_json(add_items, destination folder)

# my_query = '/search/?biosample.biosource.individual.organism.name=mouse&biosample.biosource_summary=ES-E14&experiment_type=in%20situ%20Hi-C&type=ExperimentHiC'
# store = get_query_or_linked(my_key, query=my_query)
# print(store.keys())
# print(len([i['uuid'] for key in store for i in store[key]]))
# print()

find_linked = ['b51eb447-e41e-4fe1-9004-cd47392eda46', '942e5220-825d-4379-b9b8-e4d7a5098130']
store = get_query_or_linked(my_key, linked=find_linked, linked_frame='raw')
print(store.keys())
print(len([i['uuid'] for key in store for i in store[key]]))

WORKING ON https://data.4dnucleome.org 

dict_keys(['file_fastq', 'lab', 'individual_human', 'biosample_cell_culture', 'static_section', 'user', 'experiment_set_replicate', 'target', 'file_format', 'protocol', 'biosource', 'ontology', 'vendor', 'organism', 'award', 'ontology_term', 'antibody', 'experiment_seq', 'quality_metric_fastqc', 'biosample'])
76


In [2]:
#### This part should only run once!

transfer_env = 'fourfront-webdev'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)
# reverse lookup dictionary for schema names

rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            # TODO check with all identifiers
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        # skip the items that exists
        if exists and existing:
            remove_existing_items.append(an_item['uuid'])
            print("{} {} can not post item".format(obj_type, an_item['uuid']))
            continue
 
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(len(second_round_items[a_type]), 'items transfered to target')
    print()

    

user
13 items exist on source
user 986b362f-4eb6-4a9c-8173-3ab267307e3a can not post item
user 7b8aa7bc-eee0-4bc4-9892-1c46bc4552c5 can not post item
user 986b362f-4eb6-4a9c-8173-3ab267228139 can not post item
user 7c6184f0-3198-41d2-bb71-e7af0a9a74f0 can not post item
user 32c70fcf-f3af-4687-89e2-9bdd05cf460a can not post item
user 900c5bf0-5b3a-4d7c-a6e6-7918faa19a77 can not post item
user 83b5073a-069b-4162-9b30-6f42d5551e34 can not post item
user b2ceeab8-b15b-4fdd-bcf7-49aaca674a63 can not post item
user 986b362f-4eb6-4a9c-8173-3ab267111888 can not post item
user e2324f87-0625-4bbc-803b-d47677aebe08 can not post item
3 items transfered to target

award
6 items exist on source
award a725c2ce-5be2-4933-bf56-f0b7b3f1c584 can not post item
award b0b9c607-f8b4-4f02-93f4-9895b461334b can not post item
award ae6c618f-7a8c-441e-a886-e30bbbe591da can not post item
award 43fc2984-a074-49c4-a69f-c9d3f3eadeba can not post item
2 items transfered to target

lab
8 items exist on source
lab 6240

In [5]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        counter += 1
        if a_type == 'file_fastq':
            if 'extra_files' in an_item:
                del an_item['extra_files']
        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)

In [6]:
# Round III - move attachments
import boto3
s3 = boto3.resource('s3')

#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_key)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']

#target_addresses
target_health = ff_utils.get_metadata('/health', key = transfer_key)
target_raw = target_health['file_upload_bucket'] 
target_pf = target_health['processed_file_bucket'] 
target_att = target_health['blob_bucket'] 

# Round III - move attachments
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    for an_item in second_round_items[a_type]:
        if 'attachment' in an_item.keys():
            at_key = an_item['attachment']['blob_id']
            copy_source = {'Bucket': source_att, 'Key': at_key}
            try:
                s3.meta.client.copy(copy_source, target_att, at_key)
            except:
                print('Can not find attachment on source', an_item['uuid'])
                continue
            print('attachment copied')


attachment copied
attachment copied


In [7]:
# Round IV - move files
for a_type in my_types:
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
    else:
        continue
        
    for an_item in second_round_items[a_type]:
        # accumulate all keys from a file object to be uploaded
        files_to_upload = []
        file_resp = ff_utils.get_metadata(an_item['uuid'], key = my_key)
        # add extra file keys
        if file_resp.get('extra_files', []):
            for an_extra_file in file_resp['extra_files']:
                files_to_upload.append(an_extra_file['upload_key'])
        # add main file key
        files_to_upload.append(file_resp['upload_key'])
        
        for file_key in files_to_upload:
            copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
            try:
                s3.meta.client.copy(copy_source, target_file_bucket, file_key)
            except:
                print('Can not find file on source', file_key)
                continue
            print('file copied')
            


file copied
file copied
file copied
file copied
file copied
file copied
